In [7]:
# Import Necessary Libraries
import tensorflow as tf
from keras.layers import Layer
from keras.models import load_model
import os
import glob
import csv

In [2]:
# Define Adaptive Convolutional Layer

# Initialize expected input for TF model
inputs = tf.keras.Input(shape=(150, 150, 3))

# The following class accepts an initial filter and kernel size and conducts the adaptive convolution operation on the input
class AdaptiveConvLayer(tf.keras.layers.Layer):
    def __init__(self, filters, kernel_size):
        super(AdaptiveConvLayer, self).__init__()
        self.filters = filters
        self.kernel_size = kernel_size

    def build(self, input_shape):
        self.conv_weights = self.add_weight(shape=(self.kernel_size, self.kernel_size, input_shape[-1], self.filters),
                                            initializer='random_normal',
                                            trainable=True)
        super(AdaptiveConvLayer, self).build(input_shape)

    def call(self, inputs):
        output = tf.nn.conv2d(inputs, self.conv_weights, strides=[1, 1, 1, 1], padding='SAME')
        return output

    # Generate Kernel Weights for Iteration and Return to Model
    def get_kernel_weights(self):
        return self.conv_weights


In [3]:
# Define the CustomModel class

# Global function for running the ACDA algorithm. The third function, 'get_kernel_weights' returns the kernel weights over each epoch.
class CustomModel(tf.keras.Model):
    def __init__(self):
        super(CustomModel, self).__init__()
        self.adaptive_conv = AdaptiveConvLayer(filters=3, kernel_size=3)
        self.flatten = tf.keras.layers.Flatten()
        self.dense = tf.keras.layers.Dense(6, activation='softmax')

    def call(self, inputs):
        x = self.adaptive_conv(inputs)
        x = self.flatten(x)
        x = self.dense(x)
        return x

    def get_kernel_weights(self):
        return self.adaptive_conv.get_kernel_weights()




In [4]:
# Define a custom callback to save kernel weights after each epoch

# The following function saves the returned kernel weights and saves them to a csv file for analysis.
class SaveKernelWeightsCallback(tf.keras.callbacks.Callback):
    def __init__(self, model, csv_writer):
        super(SaveKernelWeightsCallback, self).__init__()
        self.model = model
        self.csv_writer = csv_writer

    def on_epoch_end(self, epoch, logs=None):
        # Get the kernel weights of the AdaptiveConvLayer
        kernel_weights = self.model.get_kernel_weights()

        # Flatten the kernel weights for writing to csv
        flattened_weights = kernel_weights.numpy().flatten()

        # Prepare the row to write to csv
        row = [epoch] + list(flattened_weights)

        # Write the row to the csv file
        self.csv_writer.writerow(row)

In [5]:
# Create an instance of the custom model
model = CustomModel()

In [9]:
# Initialize the csv file and writer
csv_file = open('weights.csv', mode='w', newline='')
csv_writer = csv.writer(csv_file)

# Write the header row in the csv file
header = ['Epoch'] + [f'Weight_{i+1}' for i in range(model.adaptive_conv.kernel_size**2)]
csv_writer.writerow(header)

# Create the callback to save kernel weights after each epoch
save_weights_callback = SaveKernelWeightsCallback(model, csv_writer)

# Initialize necessary parameters
train_data_dir = "seg_train" # Training Image folder
batch_size = 32
image_size = (150, 150)

# Create the training dataset variable
train_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    train_data_dir,
    batch_size=batch_size,
    image_size=image_size,
    validation_split=0.2,
    subset="training",
    seed=123
)

# Compile ACDA Model
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

# Train the model with the callback to acquire weights
model.fit(train_dataset, epochs=10, callbacks=[save_weights_callback], batch_size=32)

# Close the csv file
# Close the csv file
csv_file.close()

Found 14034 files belonging to 6 classes.
Using 11228 files for training.
Epoch 1/10
351/351 [==============================] - 15s 41ms/step - loss: 35.2231 - accuracy: 0.3988
Epoch 2/10
351/351 [==============================] - 9s 27ms/step - loss: 8.0680 - accuracy: 0.7101
Epoch 3/10
351/351 [==============================] - 9s 25ms/step - loss: 2.6433 - accuracy: 0.8362
Epoch 4/10
351/351 [==============================] - 10s 27ms/step - loss: 2.2094 - accuracy: 0.8592
Epoch 5/10
351/351 [==============================] - 10s 28ms/step - loss: 28.8464 - accuracy: 0.7279
Epoch 6/10
351/351 [==============================] - 11s 32ms/step - loss: 3.0547 - accuracy: 0.8838
Epoch 7/10
351/351 [==============================] - 9s 25ms/step - loss: 0.7220 - accuracy: 0.9575
Epoch 8/10
351/351 [==============================] - 8s 22ms/step - loss: 0.4229 - accuracy: 0.9761
Epoch 9/10
351/351 [==============================] - 10s 29ms/step - loss: 0.3742 - accuracy: 0.9814
Epoch 10/1

In [11]:
# Output Model Summary
model.summary()

Model: "custom_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 adaptive_conv_layer (Adapti  multiple                 81        
 veConvLayer)                                                    
                                                                 
 flatten (Flatten)           multiple                  0         
                                                                 
 dense (Dense)               multiple                  405006    
                                                                 
Total params: 405,087
Trainable params: 405,087
Non-trainable params: 0
_________________________________________________________________


In [12]:
# Save the model's generated weights
model.save_weights('TF_Models/ECE50024_ACDA_TF_V4.h5')